# Case Study 2 - Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

In [1]:
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv("/Users/Talha/Downloads/persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [9]:
# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir?

df["SOURCE"].nunique()

2

In [10]:
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [7]:
# Soru 3: Kaç unique PRICE vardır?

df["PRICE"].nunique()

6

In [11]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş?

df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [12]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?

df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [17]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış?

df.groupby("COUNTRY").agg({"PRICE": "sum"}).T

COUNTRY,bra,can,deu,fra,tur,usa
PRICE,51354,7730,15485,10177,15689,70225


In [21]:
# Soru 7: SOURCE türlerine göre göre satış sayıları nedir?

df.SOURCE.value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [22]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?

df.groupby("COUNTRY").agg({"PRICE": "mean"}).T

COUNTRY,bra,can,deu,fra,tur,usa
PRICE,34.32754,33.608696,34.032967,33.587459,34.78714,34.007264


In [24]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?

df.groupby("SOURCE").agg({"PRICE": "mean"}).T

SOURCE,android,ios
PRICE,34.174849,34.069102


In [30]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

df.groupby(["COUNTRY","SOURCE"]).agg({"PRICE": "mean"}).T

COUNTRY        bra                   can                   deu             \
SOURCE     android        ios    android        ios    android        ios   
PRICE    34.387029  34.222222  33.330709  33.951456  33.869888  34.268817   

COUNTRY      fra                   tur                   usa             
SOURCE   android        ios    android        ios    android        ios  
PRICE    34.3125  32.776224  36.229437  33.272727  33.760357  34.371703

In [33]:
# GÖREV 2: COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?

agg_df = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE" : "mean"})


PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

In [34]:
# GÖREV 3: Çıktıyı PRICE'a göre sıralayınız.
#############################################
# Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
# Çıktıyı agg_df olarak kaydediniz.

agg_df.sort_values('PRICE', ascending=False)

len(agg_df.columns)

1

In [35]:
# GÖREV 4: Indekste yer alan isimleri değişken ismine çeviriniz.
#############################################
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
# İpucu: reset_index()
# agg_df.reset_index(inplace=True)

agg_df = agg_df.reset_index()
agg_df.columns

Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE'], dtype='object')

In [36]:
# GÖREV 5: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.
#############################################
# Age sayısal değişkenini kategorik değişkene çeviriniz.
# Aralıkları ikna edici olacağını düşündüğünüz şekilde oluşturunuz.
# Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'

# AGE değişkeninin nerelerden bölüneceğini belirtelim:

my_bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

# Bölünen noktalara karşılık isimlendirmelerin ne olacağını ifade edelim:

mylabels = ["0_18", "19_23", "24_30", "31_40", "41_" + str(agg_df["AGE"].max())]
# mylabels = ["0_18", "19_23", "24_30", "31_40", f'41_{agg_df["AGE"].max()}']

# age'i bölelim:
pd.cut(agg_df["AGE"], bins=my_bins, labels=mylabels)

agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins=my_bins, labels=mylabels)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,female,15,38.714286,0_18
1,bra,android,female,16,35.944444,0_18
2,bra,android,female,17,35.666667,0_18
3,bra,android,female,18,32.255814,0_18
4,bra,android,female,19,35.206897,19_23


In [41]:
# GÖREV 6: Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.
#############################################
# customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
# Dikkat!
# list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
# Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18
# Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.

agg_df.drop(["AGE", "PRICE"], axis=1).values

liste = ["A", "B", "C"]

agg_df["CUSTOMERS_LEVEL_BASED"] = ["_".join(i).upper() for i in agg_df.drop(['AGE', 'PRICE'], axis=1).values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,CUSTOMERS_LEVEL_BASED
0,bra,android,female,15,38.714286,0_18,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18
1,bra,android,female,16,35.944444,0_18,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18
2,bra,android,female,17,35.666667,0_18,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18
3,bra,android,female,18,32.255814,0_18,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18
4,bra,android,female,19,35.206897,19_23,BRA_ANDROID_FEMALE_19_23_BRA_ANDROID_FEMALE_19_23


In [43]:
# Gereksiz değişkenleri çıkaralım:

agg_df = agg_df[['CUSTOMERS_LEVEL_BASED', 'PRICE']]

agg_df = agg_df.groupby('CUSTOMERS_LEVEL_BASED')['PRICE'].mean().reset_index()

agg_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23_BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30_BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40_BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66_BRA_ANDROID_FEMALE_41_66,36.737179


In [45]:
# GÖREV 7: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.
#############################################
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
# segmentleri betimleyiniz, GÖREV 7: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.
#############################################
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
# segmentleri betimleyiniz

agg_df["SEGMENT"] = pd.qcut(agg_df.PRICE, q = 4, labels = ["D","C","B","A"])
agg_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18_BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23_BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30_BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40_BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66_BRA_ANDROID_FEMALE_41_66,36.737179,A


In [65]:
agg_df.groupby("SEGMENT").agg({"PRICE":["mean","max","sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.206780  32.333333   817.789833
C        33.509674  34.077340   904.761209
B        34.999645  36.000000   944.990411
A        38.691234  45.428571  1044.663328

In [66]:
# GÖREV 8: Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == new_user]


,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT


In [69]:
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == "FRA_IOS_FEMALE_31_40"]

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
